In [1]:
import pandas as pd
import matplotlib.pyplot as plt


# Load Train Dataset

In [2]:
## load train dataset

tr_pur_str = '../dataset/train_purchases.csv'
tr_purchases = pd.read_csv(tr_pur_str)

tr_ses_str = '../dataset/train_sessions.csv'
tr_sessions = pd.read_csv(tr_ses_str)


print("train purchases ")
print(tr_purchases.head(), tr_purchases.shape, len(tr_purchases['session_id'].unique()))
print(tr_purchases['item_id'].max())
print("train sessions")
print(tr_sessions.head(), tr_sessions.shape, len(tr_sessions['session_id'].unique()))
print(tr_sessions['item_id'].max())

train purchases 
   session_id  item_id                     date
0           3    15085  2020-12-18 21:26:47.986
1          13    18626  2020-03-13 19:36:15.507
2          18    24911  2020-08-26 19:20:32.049
3          19    12534   2020-11-02 17:16:45.92
4          24    13226  2020-02-26 18:27:44.114 (1000000, 3) 1000000
28143
train sessions
   session_id  item_id                     date
0           3     9655  2020-12-18 21:25:00.373
1           3     9655  2020-12-18 21:19:48.093
2          13    15654  2020-03-13 19:35:27.136
3          18    18316  2020-08-26 19:18:30.833
4          18     2507  2020-08-26 19:16:31.211 (4743820, 3) 1000000
28143


## Convert to DateTime and sort

In [ ]:
tr_sessions['date'] = pd.to_datetime(tr_sessions['date'], format='%Y-%m-%d %H:%M:%S')
# tr_sessions = tr_sessions.sort_values(['date'], ascending=True).groupby('session_id').to_frame()
tr_sessions = tr_sessions.groupby(['session_id']).apply(lambda x: x.sort_values(['date'], ascending=True))
print(tr_sessions.head())

In [ ]:
## save sorted session
tr_sessions.to_csv('./train_sessions_sorted.csv')

In [2]:
## load sorted session
tr_sessions = pd.read_csv('../dataset/train_sessions_sorted.csv')
print(tr_sessions.columns)

Index(['session_id', 'Unnamed: 1', 'session_id.1', 'item_id', 'date'], dtype='object')


In [ ]:
tr_sessions = tr_sessions[['session_id', 'item_id', 'date']]
tr_sessions.sort_index()
print(tr_sessions.head())

def add_row(x):
  sid = x.iloc[0:,:]['session_id']
  last_row = tr_purchases.loc[tr_purchases['session_id'] == sid]
  x.append(last_row)
tr_sessions.groupby('session_id').apply(add_row).reset_index(drop=True)
print(tr_sessions.head())

# Insert purchase log into train sesssions

In [ ]:
concat  = pd.concat((tr_sessions, tr_purchases)).sort_values(by=['session_id', 'date'], ascending=True)
print(concat.shape)
print(concat.head(n=10))
print(concat.tail(n=10))
concat.to_csv('./concat.csv')

(5743820, 3)


In [4]:
concat = pd.read_csv('./concat.csv')
concat.columns
print(concat['item_id'].nunique())
print(max(concat['item_id']))

23618
28143


In [19]:
print(tr_sessions.head(n=10))
print(tr_purchases.head(n=10))
print(tr_sessions.tail(n=10))
print(tr_purchases.tail(n=10))

   session_id  item_id                     date
0           3     9655  2020-12-18 21:19:48.093
1           3     9655  2020-12-18 21:25:00.373
2          13    15654  2020-03-13 19:35:27.136
3          18     4026  2020-08-26 19:15:47.232
4          18     2507  2020-08-26 19:16:31.211
5          18    18316  2020-08-26 19:18:30.833
6          19    19896  2020-11-02 16:30:36.378
7          19    27937  2020-11-02 16:30:48.207
8          19    12804  2020-11-02 16:31:05.749
9          19    25772  2020-11-02 16:31:18.543
   session_id  item_id                     date
0           3    15085  2020-12-18 21:26:47.986
1          13    18626  2020-03-13 19:36:15.507
2          18    24911  2020-08-26 19:20:32.049
3          19    12534   2020-11-02 17:16:45.92
4          24    13226  2020-02-26 18:27:44.114
5          28    26394  2020-05-18 12:52:09.764
6          31     8345  2021-04-20 19:46:42.594
7          36    14532  2020-06-21 10:33:22.535
8          42    11784  2021-03-01 15:17

In [30]:
tr_sessions = tr_sessions[['session_id', 'item_id', 'date']]
tr_sessions['date'] = pd.to_datetime(tr_sessions['date'], format='%Y-%m-%d %H:%M:%S')
tr_sessions_gb = tr_sessions.groupby(['session_id'])

from collections import defaultdict
dic = defaultdict(list)
for gr_name, gr in tr_sessions_gb:
  dic['session_id'].append(gr_name)
  dic['item_first_clicked'].append(gr.iloc[0,:]['item_id']) #categorical
  dic['year'].append(int(gr.iloc[0,:]['date'].year)) #numerical
  dic['month'].append(int(gr.iloc[0,:]['date'].month)) #numerical
  dic['day'].append(int(gr.iloc[0,:]['date'].day)) #numerical
  dic['hour'].append(int(gr.iloc[0,:]['date'].hour)) #numerical
  dic['weekofyear'].append(int(gr.iloc[0,:]['date'].weekofyear))
  dic['weekday'].append(int(gr.iloc[0,:]['date'].weekday())) #categorical
  dic['num_clicks'].append(gr.shape[0]) #numerical
  dic['duration'].append((gr.iloc[-1,:]['date'] - gr.iloc[0,:]['date']).total_seconds()//60)


In [31]:
pd.DataFrame(dic).to_csv('./feat_extract.csv')

# Load item features

## Generate train dataset txt file

In [6]:
id2idx = {}
item_features = pd.read_csv('../dataset/item_features.csv')
for i, item_id in enumerate(item_features.item_id.unique()):
  id2idx[item_id] = i
print(len(id2idx.keys()))
# id2idx_df = pd.DataFrame(list(id2idx.items()),columns = ['item_id','item_idx'])
# id2idx_df.to_csv('./id2idx.csv', index=False)


23691


In [9]:
## Generate Train dataset txt file
tr_purchases = pd.read_csv('../dataset/train_purchases.csv')
sids = tr_purchases['session_id'].unique()


with open('./train.txt', 'w') as f:
  for sid in sids:
    sess = tr_sessions.loc[tr_sessions['session_id'] == sid]['item_id'].tolist()
    sess = [id2idx[i] for i in sess]
    pur = id2idx[tr_purchases.loc[tr_purchases['session_id'] == sid]['item_id'].values[0]]
    sess.append(pur)
    string = ",".join(map(str,sess)) + '\n'
    f.write(string)

### groupby


In [ ]:
sess_grouped = tr_sessions.sort_values('date')
sess_grouped['id'] = tr_sessions.groupby(['session_id']).ngroup()
# sess_grouped.groups
# sess_grouped.reset_index()
sess_grouped

item_ids = tr_sessions['item_id'].unique()
print(len(item_ids))
item_id_dict = dict.fromkeys(item_ids)
for i, itemid in enumerate(item_ids):
  item_id_dict[itemid] = i
item_id_dict
print(len(item_id_dict.keys()))

sess_grouped['item_id'] = sess_grouped['item_id'].apply(lambda x: item_id_dict[x])
sess_grouped['item_id'].max()

sess_grouped.to_csv('./sess_grouped.csv')
sess_grouped.to_csv('./train.csv', columns=['id', 'item_id'], header=False, index=False)

23496
23496


# Plot session length of each sessions

In [10]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
from collections import defaultdict
dic = defaultdict(int)

# tr_sessions.groupby(['session_id']).sort_values([''])
session_id = tr_sessions['session_id'].unique()
for sid in session_id:
  length = len(tr_sessions.loc[tr_sessions['session_id'] == sid]['item_id'].to_list())
  if length < 20:
    dic[length] += 1
  else:
    dic['rest'] +=1

y = []
for k,v in dic.items():
  y.append(v)
ax.bar(dic.keys(),y)

# LOAD TEST SESSION


In [3]:
## load test session

te_leaderboard_sessions_str = './test_leaderboard_sessions.csv'
te_leaderboard_sessions = pd.read_csv('./test_leaderboard_sessions.csv')
print("test sessions")
print(te_leaderboard_sessions.head(), te_leaderboard_sessions.shape, len(te_leaderboard_sessions['session_id'].unique()))
print(te_leaderboard_sessions['item_id'].max())

te_final_sessions_str = './test_final_sessions.csv'
te_final_sessions = pd.read_csv('./test_final_sessions.csv')
print("test sessions")
print(te_final_sessions.head(), te_final_sessions.shape, len(te_final_sessions['session_id'].unique()))
print(te_final_sessions['item_id'].max())
# te_final

test sessions
   session_id  item_id                     date
0          26    19185  2021-06-16 09:53:54.158
1         200    17089  2021-06-25 12:23:40.811
2         200    17089  2021-06-25 12:24:36.631
3         200     8060  2021-06-25 12:24:41.677
4         200     4758  2021-06-25 12:24:50.692 (229354, 3) 50000
28137
test sessions
   session_id  item_id                     date
0          61    27088  2021-06-01 08:12:39.664
1          96    11693  2021-06-19 17:48:05.227
2          96    18298  2021-06-19 17:49:08.589
3          96     4738  2021-06-19 17:49:15.838
4          96      495   2021-06-19 17:49:20.88 (226138, 3) 50000
28137


In [3]:
te_leaderboard_sessions['date'] = pd.to_datetime(te_leaderboard_sessions['date'], format='%Y-%m-%d %H:%M:%S')
# tr_sessions = tr_sessions.sort_values(['date'], ascending=True).groupby('session_id').to_frame()
te_leaderboard_sessions = te_leaderboard_sessions.groupby(['session_id']).apply(lambda x: x.sort_values(['date'], ascending=True))
print(te_leaderboard_sessions.head())

te_final_sessions['date'] = pd.to_datetime(te_final_sessions['date'], format='%Y-%m-%d %H:%M:%S')
# tr_sessions = tr_sessions.sort_values(['date'], ascending=True).groupby('session_id').to_frame()
te_final_sessions = te_final_sessions.groupby(['session_id']).apply(lambda x: x.sort_values(['date'], ascending=True))
print(te_final_sessions.head())

              session_id  item_id                    date
session_id                                               
26         0          26    19185 2021-06-16 09:53:54.158
200        1         200    17089 2021-06-25 12:23:40.811
           2         200    17089 2021-06-25 12:24:36.631
           3         200     8060 2021-06-25 12:24:41.677
           4         200     4758 2021-06-25 12:24:50.692
              session_id  item_id                    date
session_id                                               
61         0          61    27088 2021-06-01 08:12:39.664
96         1          96    11693 2021-06-19 17:48:05.227
           2          96    18298 2021-06-19 17:49:08.589
           3          96     4738 2021-06-19 17:49:15.838
           4          96      495 2021-06-19 17:49:20.880


## generate test datset txt file

In [5]:
## Generate Test dataset txt file
sids = te_leaderboard_sessions['session_id'].unique()

with open('../datasets/dressipi/test.txt', 'w') as f:
  for sid in sids:
    sess = te_leaderboard_sessions.loc[te_leaderboard_sessions['session_id'] == sid]['item_id'].tolist()
    sess.append(te_leaderboard_sessions.loc[te_leaderboard_sessions['session_id'] == sid]['item_id'].values[0])
    string = ",".join(map(str,sess)) + '\n'
    f.write(string)

In [6]:
## Generate Test dataset txt file
sids = te_final_sessions['session_id'].unique()

with open('../datasets/dressipi/test_final.txt', 'w') as f:
  for sid in sids:
    sess = te_final_sessions.loc[te_final_sessions['session_id'] == sid]['item_id'].tolist()
    # sess.append(te_final_sessions.loc[te_final_sessions['session_id'] == sid]['item_id'].values[0])
    string = ",".join(map(str,sess)) + '\n'
    f.write(string)

## dataset statistics


In [ ]:

def print_statistics(fn):
  with open(fn, 'r') as f:
    session = f.readlines()

  len(session)
  session = [s.split(',') for s in session]
  li = list(map(len, session))
  print("average length ", sum(li)/len(li))
  print("maximum length", max(li))

  larger = sum(map((lambda x: x>5), li))
  print("session length larger than 5: ", larger)
  smaller = sum(map((lambda x: x<=5), li))
  print("session length smaller or equal to 5: ", smaller)

print_statistics('../datasets/dressipi/test.txt')
print_statistics('../datasets/dressipi/test_final.txt')
print_statistics('../datasets/dressipi/train.txt')




In [ ]:
sess_grouped = te_leaderboard_sessions
sess_grouped['id'] = te_leaderboard_sessions.groupby(['session_id']).ngroup()
# sess_grouped.groups
# sess_grouped.reset_index()
sess_grouped

item_ids = te_leaderboard_sessions['item_id'].unique()
print(len(item_ids))
item_id_dict = dict.fromkeys(item_ids)
for i, itemid in enumerate(item_ids):
  item_id_dict[itemid] = i
item_id_dict
print(len(item_id_dict.keys()))

sess_grouped['item_id'] = sess_grouped['item_id'].apply(lambda x: item_id_dict[x])
print(sess_grouped['item_id'].max())

sess_grouped.to_csv('./sess_grouped.csv')
sess_grouped.to_csv('./test.csv', columns=['id', 'item_id'], header=False, index=False)

5647
5647
5646


In [ ]:

def session_length(session_pd, path):
  ## get length of each session
  sess_grouped = session_pd.groupby(['session_id']).size().to_frame('size')
  # sess_grouped['size'] = sess_grouped['session_id'].transform('size')
  fn = path.split('/')[-1]
  sess_grouped.to_csv('./{}_grouped.csv'.format(fn))
  return sess_grouped

tr_sess_gr = session_length(tr_sessions, tr_ses_str)
session_length(te_leaderboard_sessions, te_leaderboard_sessions_str)
session_length(te_final_sessions, te_final_sessions_str)

,size
session_id,
61,1
96,5
185,5
224,2
285,1
...,...
4439646,4
4439648,1
4439675,1


In [ ]:
## count session length
print(tr_sess_gr['size'].value_counts())
tr_sess_gr['size'].value_counts().to_csv('./train_session_length.csv')

1     307698
2     174775
3     115499
4      82306
5      60695
       ...  
91         6
96         6
98         5
99         5
89         5
Name: size, Length: 100, dtype: int64


In [ ]:
it_feat_str = './item_features.csv'
item_features = pd.read_csv(it_feat_str)

print("item_features")
print(item_features.head(), item_features.shape)
print(len(item_features['item_id'].unique()), len(item_features['feature_category_id'].unique()), len(item_features['feature_value_id'].unique()))

item_features
   item_id  feature_category_id  feature_value_id
0        2                   56               365
1        2                   62               801
2        2                   68               351
3        2                   33               802
4        2                   72                75 (471751, 3)
23691 73 890


In [ ]:
cand_it_str = './candidate_items.csv'
candidate_items = pd.read_csv(cand_it_str)

print("candidate items")
print(candidate_items.head(), candidate_items.shape)


candidate items
   item_id
0        4
1        8
2        9
3       19
4       20 (4990, 1)
